In [ ]:
# default_exp models.tacotron2.decoder

In [ ]:
# export
import pdb
from torch import nn
from uberduck_ml_dev.models.base import TTSModel
from uberduck_ml_dev.models.common import Attention, Conv1d, LinearNorm, GST
from uberduck_ml_dev.text.symbols import symbols
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
from uberduck_ml_dev.utils.utils import to_gpu, get_mask_from_lengths
import numpy as np
import torch
from torch.autograd import Variable
from torch.cuda.amp import autocast
from torch.nn import functional as F
from uberduck_ml_dev.utils.duration import (
    GaussianUpsampling,
    RangePredictor,
    PositionalEncoding,
    DurationPredictor,
)
from uberduck_ml_dev.models.tacotron2.prenet import Prenet
from einops import rearrange
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Decoder(nn.Module):
    def __init__(self, hparams):
        super().__init__()
        self.n_mel_channels = hparams.n_mel_channels
        self.n_frames_per_step_initial = hparams.n_frames_per_step_initial
        self.n_frames_per_step_current = hparams.n_frames_per_step_initial
        self.encoder_embedding_dim = hparams.encoder_embedding_dim
        self.attention_rnn_dim = hparams.attention_rnn_dim
        self.decoder_rnn_dim = hparams.decoder_rnn_dim
        self.prenet_dim = hparams.prenet_dim
        self.max_decoder_steps = hparams.max_decoder_steps
        self.gate_threshold = hparams.gate_threshold
        self.p_attention_dropout = hparams.p_attention_dropout
        self.p_decoder_dropout = hparams.p_decoder_dropout
        self.p_teacher_forcing = hparams.p_teacher_forcing
        self.cudnn_enabled = hparams.cudnn_enabled
        self.non_attentive = hparams.non_attentive
        self.positional_embedding_dim = hparams.positional_embedding_dim
        self.decoder_rnn_dim_nlayers = hparams.decoder_rnn_dim_nlayers
        self.location_specific_attention = hparams.location_specific_attention
        if hparams.non_attentive:
            self.gaussian_upsampling = GaussianUpsampling(hparams)
            self.range_predictor = RangePredictor(hparams)
            self.duration_predictor = DurationPredictor(hparams)
        self.prenet = Prenet(
            hparams.n_mel_channels, [hparams.prenet_dim, hparams.prenet_dim],
        )

        if self.location_specific_attention:
            self.attention_rnn = nn.LSTMCell(
                hparams.prenet_dim + self.encoder_embedding_dim,
                hparams.attention_rnn_dim,
            )

            self.attention_layer = Attention(
                hparams.attention_rnn_dim,
                self.encoder_embedding_dim,
                hparams.attention_dim,
                hparams.attention_location_n_filters,
                hparams.attention_location_kernel_size,
                fp16_run=hparams.fp16_run,
            )
            self.linear_projection = LinearNorm(
                hparams.decoder_rnn_dim + self.encoder_embedding_dim,
                hparams.n_mel_channels * hparams.n_frames_per_step_initial,
            )
            self.decoder_rnn = nn.LSTMCell(
                hparams.attention_rnn_dim + self.encoder_embedding_dim,
                hparams.decoder_rnn_dim,
                1,
            )
        if self.non_attentive:
            self.linear_projection = LinearNorm(
                hparams.decoder_rnn_dim
                + hparams.encoder_embedding_dim
                + self.positional_embedding_dim,
                hparams.n_mel_channels,
            )
            self.positional_embedding = PositionalEncoding(
                hparams.positional_embedding_dim
            )

            #NOTE (SAM): https://stackoverflow.com/questions/57048120/pytorch-lstm-vs-lstmcell
            self.decoder_rnn = nn.LSTM(input_size=self.prenet_dim + self.encoder_embedding_dim + self.positional_embedding_dim,hidden_size=self.decoder_rnn_dim,num_layers=self.decoder_rnn_dim_nlayers,batch_first=True)
        # self.linear_projection2 = LinearNorm(hparams.positional_embedding_dim, hparams.n_mel_channels)
        self.gate_layer = LinearNorm(
            hparams.decoder_rnn_dim + self.encoder_embedding_dim,
            1,
            bias=True,
            w_init_gain="sigmoid",
        )

    def set_current_frames_per_step(self, n_frames: int):
        self.n_frames_per_step_current = n_frames

    def get_go_frame(self, memory):
        """Gets all zeros frames to use as first decoder input
        PARAMS
        ------
        memory: decoder outputs

        RETURNS
        -------
        decoder_input: all zeros frames
        """
        B = memory.size(0)
        decoder_input = Variable(memory.data.new(B, self.n_mel_channels).zero_())
        return decoder_input

    def get_positional_embedding(self, durations):
        """
            :param durations: [B, N]
            :return positional_embedding_outputs: [B, T, positional_hidden]
        """

        B = durations.size(0)
        N = durations.size(1)

        pos_durations = durations.long()
        sum_len = pos_durations.sum(dim=1)
        max_len = sum_len.max()
        diff_len = max_len - sum_len
        pos_durations[:, -1] = pos_durations[:, -1] + diff_len

        ids = torch.arange(max_len, device=durations.device).expand(B, N, max_len)
        pos_mask = ids < pos_durations.view(B, N, 1)
        pos_ids = ids[pos_mask].view(-1, max_len)  # [B, T]
        positional_embedding_outputs = self.positional_embedding(pos_ids)

        return positional_embedding_outputs

    def initialize_decoder_states(self, memory, mask):
        """Initializes attention rnn states, decoder rnn states, attention
        weights, attention cumulative weights, attention context, stores memory
        and stores processed memory
        PARAMS
        ------
        memory: Encoder outputs
        mask: Mask for padded data if training, expects None for inference
        """
        B = memory.size(0)
        MAX_TIME = memory.size(1)

        if self.location_specific_attention:
            self.attention_hidden = Variable(
                memory.data.new(B, self.attention_rnn_dim).zero_()
            )
            self.attention_cell = Variable(
                memory.data.new(B, self.attention_rnn_dim).zero_()
            )
            self.attention_weights = Variable(memory.data.new(B, MAX_TIME).zero_())
            self.attention_weights_cum = Variable(memory.data.new(B, MAX_TIME).zero_())
            self.attention_context = Variable(
                memory.data.new(B, self.encoder_embedding_dim).zero_()
            )
            self.processed_memory = self.attention_layer.memory_layer(memory)
            self.decoder_hidden = Variable(memory.data.new(B, self.decoder_rnn_dim).zero_())
            self.decoder_cell = Variable(memory.data.new(B, self.decoder_rnnself.decoder_rnn_dim).zero_())
            
        if self.non_attentive:
            self.decoder_hidden = Variable(memory.data.new(B, self.decoder_rnn_dim_nlayers, self.decoder_rnn_dim).zero_())


        self.memory = memory

        self.mask = mask

    def parse_decoder_inputs(self, decoder_inputs):
        """Prepares decoder inputs, i.e. mel outputs
        PARAMS
        ------
        decoder_inputs: inputs used for teacher-forced training, i.e. mel-specs

        RETURNS
        -------
        inputs: processed decoder inputs

        """
        # NOTE (SAM): Rewrite with einops rearrange
        # (B, n_mel_channels, T_out) -> (B, T_out, n_mel_channels)

        decoder_inputs = decoder_inputs.transpose(1, 2)
        decoder_inputs = decoder_inputs.contiguous()
        # print(decoder_inputs.shape, 'dec_in')
        # print(decoder_inputs.size(0), int(decoder_inputs.size(1)), self.n_frames_per_step_current)
        decoder_inputs = decoder_inputs.view(
            decoder_inputs.size(0),
            int(decoder_inputs.size(1) / self.n_frames_per_step_current),
            -1,
        )
        # (B, T_out, n_mel_channels) -> (T_out, B, n_mel_channels)
        decoder_inputs = decoder_inputs.transpose(0, 1)
        return decoder_inputs

    def parse_decoder_outputs(self, mel_outputs, gate_outputs, alignments):
        """Prepares decoder outputs for output
        PARAMS
        ------
        mel_outputs:
        gate_outputs: gate output energies
        alignments:

        RETURNS
        -------
        mel_outputs:
        gate_outpust: gate output energies
        alignments:
        """
        # (T_out, B) -> (B, T_out)
        alignments = torch.stack(alignments).transpose(0, 1)
        # (T_out, B) -> (B, T_out)
        gate_outputs = torch.stack(gate_outputs)
        if len(gate_outputs.size()) > 1:
            gate_outputs = gate_outputs.transpose(0, 1)
        else:
            gate_outputs = gate_outputs[None]
        gate_outputs = gate_outputs.contiguous()
        # (B, T_out, n_mel_channels * n_frames_per_step) -> (B, T_out * n_frames_per_step, n_mel_channels)
        mel_outputs = mel_outputs.view(mel_outputs.size(0), -1, self.n_mel_channels)
        # (B, T_out, n_mel_channels) -> (B, n_mel_channels, T_out)
        mel_outputs = mel_outputs.transpose(1, 2)

        return mel_outputs, gate_outputs, alignments

    def decode(self, decoder_input, attention_weights=None):
        """Decoder step using stored states, attention and memory
        PARAMS
        ------
        decoder_input: previous mel output

        RETURNS
        -------
        mel_output:
        gate_output: gate output energies
        attention_weights:
        """

        cell_input = torch.cat((decoder_input, self.attention_context), -1)

        self.attention_hidden, self.attention_cell = self.attention_rnn(
            cell_input, (self.attention_hidden, self.attention_cell)
        )
        self.attention_hidden = F.dropout(
            self.attention_hidden, self.p_attention_dropout, self.training
        )
        self.attention_cell = F.dropout(
            self.attention_cell, self.p_attention_dropout, self.training
        )

        attention_weights_cat = torch.cat(
            (
                self.attention_weights.unsqueeze(1),
                self.attention_weights_cum.unsqueeze(1),
            ),
            dim=1,
        )
        self.attention_context, self.attention_weights = self.attention_layer(
            self.attention_hidden,
            self.memory,
            self.processed_memory,
            attention_weights_cat,
            self.mask,
            attention_weights,
        )

        self.attention_weights_cum += self.attention_weights

        decoder_input = torch.cat((self.attention_hidden, self.attention_context), -1)
        self.decoder_hidden, self.decoder_cell = self.decoder_rnn(
            decoder_input, (self.decoder_hidden, self.decoder_cell)
        )
        self.decoder_hidden = F.dropout(
            self.decoder_hidden, self.p_decoder_dropout, self.training
        )
        self.decoder_cell = F.dropout(
            self.decoder_cell, self.p_decoder_dropout, self.training
        )

        decoder_hidden_attention_context = torch.cat(
            (self.decoder_hidden, self.attention_context), dim=1
        )

        decoder_output = self.linear_projection(decoder_hidden_attention_context)

        gate_prediction = self.gate_layer(decoder_hidden_attention_context)
        return decoder_output, gate_prediction, self.attention_weights

    def forward(
        self,
        encoder_outputs,
        decoder_inputs,
        output_lengths,
        input_lengths=None,
        durations=None,
    ):
        """Decoder forward pass for training
        PARAMS
        ------
        encoder_outputs: Encoder outputs
        decoder_inputs: Decoder inputs for teacher forcing. i.e. mel-specs
        output_lengths: Encoder output lengths for attention masking.
        input_lenghts: phrase lengths ntokens (same as inut lengths?)

        RETURNS
        -------
        mel_outputs: mel outputs from the decoder
        gate_outputs: gate outputs from the decoder
        alignments: sequence of attention weights from the decoder
        """
        B = encoder_outputs.size(0)
        mel_outputs = torch.empty(
            B, 0, self.n_frames_per_step_current * self.n_mel_channels
        )
        if torch.cuda.is_available() and self.cudnn_enabled:
            mel_outputs = mel_outputs.cuda()

        if self.location_specific_attention:

            #pdb.set_trace()  # NOTE (SAM): FIX THIS USING EINOPS REARRANGE TO NOT BE INSANE

            decoder_inputs = self.parse_decoder_inputs(decoder_inputs)
            decoder_inputs = decoder_inputs.reshape(
                -1, decoder_inputs.size(1), self.n_mel_channels
            )
            decoder_input = self.get_go_frame(encoder_outputs).unsqueeze(0)
            decoder_inputs = torch.cat((decoder_input, decoder_inputs), dim=0)
            decoder_inputs = self.prenet(decoder_inputs)

            self.initialize_decoder_states(
                encoder_outputs, mask=~get_mask_from_lengths(output_lengths)
            )

            gate_outputs, alignments = [], []
            desired_output_frames = (
                decoder_inputs.size(0) / self.n_frames_per_step_current
            )
            while mel_outputs.size(1) < desired_output_frames - 1:
                if (
                    mel_outputs.size(1) == 0
                    or np.random.uniform(0.0, 1.0) <= self.p_teacher_forcing
                ):
                    teacher_forced_frame = decoder_inputs[
                        mel_outputs.size(1) * self.n_frames_per_step_current
                    ]

                    to_concat = (teacher_forced_frame,)
                    decoder_input = torch.cat(to_concat, dim=1)
                else:

                    # NOTE(zach): we may need to concat these as we go to ensure that
                    # it's easy to retrieve the last n_frames_per_step_init frames.
                    to_concat = (
                        self.prenet(
                            mel_outputs[:, -1, -1 * self.n_frames_per_step_current :]
                        ),
                    )
                    decoder_input = torch.cat(to_concat, dim=1)
                # NOTE(zach): When training with fp16_run == True, decoder_rnn seems to run into
                # issues with NaNs in gradient, maybe due to vanishing gradients.
                # Disable half-precision for this call to work around the issue.
                with autocast(enabled=False):
                    mel_output, gate_output, attention_weights = self.decode(
                        decoder_input
                    )
                mel_outputs = torch.cat(
                    [
                        mel_outputs,
                        mel_output[
                            :, 0 : self.n_mel_channels * self.n_frames_per_step_current
                        ].unsqueeze(1),
                    ],
                    dim=1,
                )
                gate_outputs += [gate_output.squeeze()] * self.n_frames_per_step_current
                alignments += [attention_weights]

        if self.non_attentive:
            
            self.initialize_decoder_states(
                encoder_outputs, mask=~get_mask_from_lengths(output_lengths)
            )
            predicted_durations = self.duration_predictor(
                encoder_outputs, input_lengths.cpu()
            )
            range_outputs = self.range_predictor(
                encoder_outputs, durations, input_lengths.cpu()
            )

            encoder_upsampling_outputs = self.gaussian_upsampling(
                encoder_outputs, durations, range_outputs, input_lengths
            )

            positional_embedding_outputs = self.get_positional_embedding(durations)
            encoder_concated_outputs = torch.cat(
                [encoder_upsampling_outputs, positional_embedding_outputs], dim=2
            )
            #pdb.set_trace()

            guided_mel_specs = decoder_inputs[:, :, :-1]
            dummy_spec = self.get_go_frame(encoder_outputs).unsqueeze(2)
            guided_mel_specs = torch.cat([dummy_spec, guided_mel_specs], dim=2)
            guided_mel_specs = rearrange(guided_mel_specs, "b m o -> b o m")
            # guided_mel_specs = guided_mel_specs.transpose(1, 2)
            prenet_outputs = self.prenet(guided_mel_specs)
            concated_prenet_outputs = torch.cat(
                [prenet_outputs, encoder_concated_outputs], dim=2
            )
            # NOTE (Sam): durations should sum to target lenghts
            target_lengths= durations.long().sum(dim=1).detach().cpu()
            packed_prenet_outputs = pack_padded_sequence(concated_prenet_outputs,target_lengths,batch_first=True,enforce_sorted=False,)
            
            #self.decoder_rnn.flatten_parameters()
            decoder_lstm_outputs, _ =  self.decoder_rnn(concated_prenet_outputs)
            #decoder_lstm_outputs, _ = self.decoder_rnn(concated_prenet_outputs,(self.decoder_hidden, self.decoder_cell))  # [B, T, decoder_rnn_dim]
            decoder_lstm_outputs, _ = pad_packed_sequence(
                decoder_lstm_outputs, batch_first=True
            )
            pdb.set_trace()
            concated_decoder_lstm_outputs = torch.cat(
                [decoder_lstm_outputs, encoder_concated_outputs], dim=2
            )  # [B, T, decoder_lstm_dim + encoder_lstm_dim + positional_embedding_dim]
            predicted_mel_specs = self.linear_projection(
                concated_decoder_lstm_outputs
            )  # [B, T, n_mel_channels]
            predicted_mel_specs = predicted_mel_specs.transpose(
                1, 2
            )  # [B, n_mel_channels, T]

        mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
            mel_outputs, gate_outputs, alignments
        )

        return mel_outputs, gate_outputs, alignments

    def inference(self, memory, input_lengths):
        """Decoder inference
        PARAMS
        ------
        memory: Encoder outputs
        input_lengths (nee memory_lenghts): encoder input_lengths

        RETURNS
        -------
        mel_outputs: mel outputs from the decoder
        gate_outputs: gate outputs from the decoder
        alignments: sequence of attention weights from the decoder
        """
        decoder_input = self.get_go_frame(memory)
        self.initialize_decoder_states(
            memory, mask=~get_mask_from_lengths(input_lengths)
        )

        B = memory.size(0)
        mel_outputs = torch.empty(
            B, 0, self.n_frames_per_step_current * self.n_mel_channels
        )
        if torch.cuda.is_available() and self.cudnn_enabled:
            mel_outputs = mel_outputs.cuda()
        gate_outputs, alignments = [], []

        mel_lengths = torch.zeros(
            [memory.size(0)], dtype=torch.int32, device=memory.device
        )
        not_finished = torch.ones(
            [memory.size(0)], dtype=torch.int32, device=memory.device
        )

        while True:
            to_cat = (self.prenet(decoder_input),)

            decoder_input = torch.cat(to_cat, dim=1)
            mel_output, gate_output, alignment = self.decode(decoder_input)
            mel_output = mel_output[
                :, 0 : self.n_mel_channels * self.n_frames_per_step_current
            ].unsqueeze(1)

            mel_outputs = torch.cat([mel_outputs, mel_output], dim=1)
            gate_outputs += [gate_output.squeeze()] * self.n_frames_per_step_current
            alignments += [alignment]

            dec = (
                torch.le(torch.sigmoid(gate_output), self.gate_threshold)
                .to(torch.int32)
                .squeeze(1)
            )

            not_finished = not_finished * dec
            mel_lengths += not_finished

            if torch.sum(not_finished) == 0:
                break
            if mel_outputs.shape[1] == self.max_decoder_steps:
                print("Warning! Reached max decoder steps")
                break

            decoder_input = mel_output[:, -1, -1 * self.n_mel_channels :]
        mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
            mel_outputs, gate_outputs, alignments
        )

        return mel_outputs, gate_outputs, alignments, mel_lengths

    def inference_noattention(self, memory, attention_map):
        """Decoder inference
        PARAMS
        ------
        memory: Encoder outputs

        RETURNS
        -------
        mel_outputs: mel outputs from the decoder
        gate_outputs: gate outputs from the decoder
        alignments: sequence of attention weights from the decoder
        """
        decoder_input = self.get_go_frame(memory)

        self.initialize_decoder_states(memory, mask=None)

        B = memory.size(0)
        mel_outputs = torch.empty(
            B, 0, self.n_frames_per_step_current * self.n_mel_channels
        )
        if torch.cuda.is_available() and self.cudnn_enabled:
            mel_outputs = mel_outputs.cuda()
        gate_outputs, alignments = [], []
        for i in range(len(attention_map)):

            attention = attention_map[i]
            decoder_input = self.prenet(decoder_input)
            mel_output, gate_output, alignment = self.decode(decoder_input, attention)
            mel_output = mel_output[
                :, 0 : self.n_mel_channels * self.n_frames_per_step_current
            ].unsqueeze(1)

            mel_outputs = torch.cat([mel_outputs, mel_output], dim=1)
            gate_outputs += [gate_output.squeeze()] * self.n_frames_per_step_current
            alignments += [alignment]

            decoder_input = mel_output[:, -1, -1 * self.n_mel_channels :]

        mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
            mel_outputs, gate_outputs, alignments
        )

        return mel_outputs, gate_outputs, alignments

    def inference_partial_tf(self, memory, decoder_inputs, tf_until_idx, device="cpu"):
        """Decoder inference with teacher-forcing up until tf_until_idx
        PARAMS
        ------
        memory: Encoder outputs
        decoder_inputs: Decoder inputs for teacher forcing. i.e. mel-specs
        memory_lengths: Encoder output lengths (decoder input lengths) for attention masking.

        RETURNS
        -------
        mel_outputs: mel outputs from the decoder
        gate_outputs: gate outputs from the decoder
        alignments: sequence of attention weights from the decoder
        """
        if device == "cuda" and torch.cuda.is_available() and self.cudnn_enabled:
            decoder_inputs = decoder_inputs.cuda()

        B = memory.size(0)
        decoder_inputs = self.parse_decoder_inputs(decoder_inputs)
        decoder_inputs = decoder_inputs.reshape(
            -1, decoder_inputs.size(1), self.n_mel_channels
        )
        decoder_input = self.get_go_frame(memory).unsqueeze(0)
        decoder_inputs = torch.cat((decoder_input, decoder_inputs), dim=0)
        decoder_inputs = self.prenet(decoder_inputs)

        self.initialize_decoder_states(memory, mask=None)

        mel_outputs = torch.empty(
            B, 0, self.n_frames_per_step_current * self.n_mel_channels
        )
        if device == "cuda" and torch.cuda.is_available() and self.cudnn_enabled:
            mel_outputs = mel_outputs.cuda()
        gate_outputs, alignments = [], []

        while True:
            if mel_outputs.size(1) < tf_until_idx:
                teacher_forced_frame = decoder_inputs[
                    mel_outputs.size(1) * self.n_frames_per_step_current
                ]

                to_concat = (teacher_forced_frame,)
                decoder_input = torch.cat(to_concat, dim=1)
            else:

                to_concat = (
                    self.prenet(mel_outputs[:, -1, -1 * self.n_mel_channels :]),
                )
                decoder_input = torch.cat(to_concat, dim=1)
            mel_output, gate_output, attention_weights = self.decode(decoder_input)
            mel_outputs = torch.cat(
                [
                    mel_outputs,
                    mel_output[
                        :, 0 : self.n_mel_channels * self.n_frames_per_step_current
                    ].unsqueeze(1),
                ],
                dim=1,
            )
            gate_outputs += [gate_output.squeeze()] * self.n_frames_per_step_current
            alignments += [attention_weights]
            if torch.sigmoid(gate_output.data) > self.gate_threshold:
                break
            elif mel_outputs.size(1) == self.max_decoder_steps:
                print("Warning! Reached max decoder steps")
                break

        mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
            mel_outputs, gate_outputs, alignments
        )

        return mel_outputs, gate_outputs, alignments